In [1]:
# %pip install pypdf
# %pip install chromadb
# %pip install --upgrade chromadb

In [4]:
from langchain.document_loaders import PyPDFLoader

file_path = 'mixed_data/element_of_SL.pdf'

loader = PyPDFLoader(file_path=file_path)

docs = loader.load_and_split()
docs

[Document(metadata={'source': 'mixed_data/element_of_SL.pdf', 'page': 0}, page_content='Springer Series in Statistics\nTrevor Hastie\nRobert TibshiraniJerome FriedmanSpringer Series in Statistics\nThe Elements of\nStatistical Learning\nData Mining, Inference, and Prediction\nThe Elements of Statistical LearningDuring the past decade there has been an explosion in computation and information tech-\nnology. With it have come vast amounts of data in a variety of fields such as medicine, biolo-gy, finance, and marketing. The challenge of understanding these data has led to the devel-opment of new tools in the field of statistics, and spawned new areas such as data mining,machine learning, and bioinformatics. Many of these tools have common underpinnings butare often expressed with different terminology. This book describes the important ideas inthese areas in a common conceptual framework. While the approach is statistical, theemphasis is on concepts rather than mathematics. Many examples 

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

index = Chroma.from_documents(
    docs,
    embedding = OpenAIEmbeddings()
)

C:\Users\Owner\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Stuff strategy

In [7]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler

llm = ChatOpenAI()

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=index.as_retriever(),
    chain_type='stuff',
    verbose=True
)

chain.run(
    'What is linear regression?',
    callbacks=[StdOutCallbackHandler()]
)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
100 3. Linear Methods for Regression

This is page 43
Printer: Opaque this
3
Linear Methods for Regression
3.1 Introduction
A linear regression model assumes that the regression function E( Y|X) is
linear in the inputs X1,... ,X p. Linear models were largely developed in
the precomputer age of statistics, but even in today’s computer era there
are still good reasons to study and use them. They are simple and often
provide an adequate and interpretable description of how the inputs aﬀect
the output. For prediction purposes they can sometimes outperform fancier
nonlinear models, especially in situations with small numbers of training
cases, low signal-to-noise rati

'Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. The linear regression model assumes that the relationship between the variables can be represented by a linear equation. This model aims to find the best-fitting straight line that predicts the dependent variable based on the independent variable(s). The parameters of the linear regression model are estimated using techniques like least squares, where the goal is to minimize the sum of the squared differences between the observed values and the values predicted by the model.'

## The map-reduce chain

In [8]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type='map_reduce',
    verbose=True
)

chain.run(
    'What is linear regression?',
    callbacks=[StdOutCallbackHandler()]
)



> Entering new RetrievalQA chain...


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
100 3. Linear Methods for Regression
Human: What is linear regression?
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
This is page 43
Printer: Opaque this
3
Linear Methods for Regression
3.1 Introduction
A linear regression model assumes that the regression function E( Y|X) is
linear in the inputs X1,... ,X p. Linear models were largely developed in
the precomputer age of statistics, but even in today’s computer era there
are still good reasons to study and use them. They are simple and often
provide an adequate and inte

'Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to the observed data. It assumes that the relationship between the variables is linear and aims to find the best-fitting line that describes this relationship.'

## The refine strategy

In [11]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type='refine',
    verbose=True
)

chain.run(
    'What is linear regression?',
    callbacks=[StdOutCallbackHandler()]
)



> Entering new RetrievalQA chain...


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Context information is below.
------------
100 3. Linear Methods for Regression
------------
Given the context information and not prior knowledge, answer any questions
Human: What is linear regression?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: What is linear regression?
AI: Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to observed data. The goal of linear regression is to find the best-fitting line that represents the relationship between the variables, allowing us to make predictions or understand the impact of the independent variables on the dependent variable.
Human: We have the opportunity to refine the existing answer (only if needed) with some more context below.


'The additional context provided does not require any changes to the original answer.'

## The map-rerank strategy

In [12]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.as_retriever(),
    chain_type='map_rerank',
    verbose=True
)

chain.run(
    'What is linear regression?',
    callbacks=[StdOutCallbackHandler()]
)



> Entering new RetrievalQA chain...


> Entering new MapRerankDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not su

C:\Users\Owner\anaconda3\envs\langchain\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


'Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to observed data. This method is commonly used for predicting future values based on past data. '